In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<!-- <table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/notebook_template.ipynb"">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table> -->

#  Vertex Pipelines: Pytorch resnet CIFAR10 e2e example

## Overview

Pytorch resnet CIFAR10 e2e example using [Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines).

This `kfp.v2` code currently does not run on OSS KFP, but will be able to soon once the 'v2 compatibility mode' feature is supported.


### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages



In [2]:
PROJECT_ID = 'rthallam-demo-project'  # <---CHANGE THIS

Project ID: rthallam-demo-project


In [2]:
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


On colab, authenticate first:

In [3]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

Then, install the libraries.

In [4]:
import sys
if 'google.colab' in sys.modules:
  USER_FLAG = ''
else:
  USER_FLAG = '--user'

In [ ]:
!python3 -m pip install {USER_FLAG} torch sklearn webdataset torchvision pytorch-lightning boto3 google-cloud-build --upgrade

In [ ]:
!python3 -m pip install {USER_FLAG} kfp google-cloud-pipeline-components --upgrade

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [7]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

Check the versions of the packages you installed.  The KFP SDK version should be >=1.6.

In [1]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"

KFP SDK version: 1.6.2


## Before you begin

This notebook does not require a GPU runtime.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component). 
Also [enable the Cloud Build API](https://console.cloud.google.com/flows/enableapi?apiid=cloudbuild.googleapis.com).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
import os
# Get your Google Cloud project ID from gcloud
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null

try:
    PROJECT_ID = shell_output[0]
except IndexError:
    PROJECT_ID = None

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  rthallam-demo-project


Otherwise, set your project ID here.

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "python-docs-samples-tests"  # @param {type:"string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [3]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "AI Platform"
into the filter box, and select
   **AI Platform Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [4]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AI Platform, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket as necessary

You will need a Cloud Storage bucket for this example.  If you don't have one that you want to use, you can make one now.


Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where AI Platform (Unified) services are
available](https://cloud.google.com/ai-platform-unified/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with AI Platform.

**Change the bucket name below** before running the next cell.

In [5]:
BUCKET_NAME = "gs://cloud-ai-platform-2f444b6a-a742-444b-b91a-c7519f51bd77"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

In [6]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

### Import libraries and define constants



Define some constants. See the "Before you begin" section of the Managed Pipelines User Guide for information on creating your API key.


In [7]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

USER = 'rthallam' # <---CHANGE THIS
PIPELINE_ROOT = '{}/pipeline_root/{}'.format(BUCKET_NAME, USER)

PIPELINE_ROOT

env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


'gs://cloud-ai-platform-2f444b6a-a742-444b-b91a-c7519f51bd77/pipeline_root/rthallam'

Do some imports:

In [8]:
import json
from typing import NamedTuple


from kfp import dsl
from kfp.v2 import compiler
from typing import NamedTuple
from kfp.v2 import dsl
from kfp.v2.dsl import (
    component,
    InputPath,
    OutputPath,
    Input,
    Output,
    Artifact,
    Dataset,
    Model,
    ClassificationMetrics,
    Metrics,
)

from kfp.v2.google.client import AIPlatformClient

from google_cloud_pipeline_components import aiplatform as gcc_aip

## Define the pipeline components



### (Optional, not necessary to run the example) Create base images from modified version of repo code

To run the example, you can just use the public images below.  Here's the info on building the images.

Download and unzip the zip file [in this folder](https://drive.google.com/drive/folders/1_rX4UaI3h4rcciJhZcyJA2CatzfwNw_r?usp=sharing), and build an image using the `Dockerfile` in the top-level directory, then push it to gcr.
Then  build an image for `Dockerfile-gpu`.

What's currently different from the repo version (at time of copy):
- changed (and moved) the Dockerfile, added a GPU version of the dockerfile.
- copied these two files to the pytorch-pipelines top-level dir: `cifar10_train.py`, `cifar10_datamodule.py`
- changed one file, `pytorch_pipeline/components/trainer/executor.py`, according to [this patch](https://drive.google.com/file/d/1djdMxroHKu9O024_Eq8zU-RM_Q77zPXK/view?usp=sharing), to get rid of the minio stuff

**Edit the next cell** to point to your image builds.

In [9]:
CONTAINER_URI = "gcr.io/google-samples/pytorch-pl:v1"
GPU_CONTAINER_URI = "gcr.io/google-samples/pytorch-pl-gpu:v1"

### Define the 'preprocess' component

The preproc component is an adapted version of `examples/cifar10/cifar10_pre_process.py`.

In [10]:
@component(
    base_image=CONTAINER_URI,
    output_component_file="cifar_preproc.yaml",
)
def cifar_preproc(
    cifar_dataset: Output[Dataset],
):

    import subprocess
    import logging
    from pathlib import Path

    import torchvision
    import webdataset as wds
    from sklearn.model_selection import train_test_split

    logging.getLogger().setLevel(logging.INFO)
    logging.info("Dataset path is: %s", cifar_dataset.path)
    output_pth = cifar_dataset.path

    Path(output_pth).mkdir(parents=True, exist_ok=True)

    trainset = torchvision.datasets.CIFAR10(
        root="./", train=True, download=True
    )
    testset = torchvision.datasets.CIFAR10(
        root="./", train=False, download=True
    )

    Path(output_pth + "/train").mkdir(parents=True, exist_ok=True)
    Path(output_pth + "/val").mkdir(parents=True, exist_ok=True)
    Path(output_pth + "/test").mkdir(parents=True, exist_ok=True)

    random_seed = 25
    y = trainset.targets
    trainset, valset, y_train, y_val = train_test_split(
        trainset,
        y,
        stratify=y,
        shuffle=True,
        test_size=0.2,
        random_state=random_seed,
    )

    for name in [(trainset, "train"), (valset, "val"), (testset, "test")]:
        with wds.ShardWriter(
            output_pth + "/" + str(name[1]) + "/" + str(name[1]) + "-%d.tar",
            maxcount=1000,
        ) as sink:
            for index, (image, cls) in enumerate(name[0]):
                sink.write(
                    {"__key__": "%06d" % index, "ppm": image, "cls": cls}
                )

    entry_point = ["ls", "-R", output_pth]
    run_code = subprocess.run(entry_point, stdout=subprocess.PIPE)
    print(run_code.stdout)


### Define a component to create torchserve `Dockerfile` and `config.properties` files from the pipeline params

(**TODO**: a more robust approach would be to read from and modify template files rather than defining them inline).

This version uses the `json` service envelope.  

**TODO**: use `kfserving` service envelope. For some reason that version failed to deploy successfully to ucaip prediction.

In [11]:
@component(
    # base_image=CONTAINER_URI,
    output_component_file="cifar_config.yaml",
)
def cifar_config(
    mar_model_name: str,
    version: str,
    port: int,
    cifar_config: Output[Artifact],
):
    import os
    from pathlib import Path
  
    Path(cifar_config.path).mkdir(parents=True, exist_ok=True)

    config_properties = f"""inference_address=http://0.0.0.0:{port}
management_address=http://0.0.0.0:8081
metrics_address=http://0.0.0.0:8082
enable_metrics_api=true
metrics_format=prometheus
number_of_netty_threads=4
job_queue_size=10
service_envelope=kfserving
model_store=/home/model-server/model-store
model_snapshot={{"name":"startup.cfg","modelCount":1,"models":{{"{mar_model_name}":{{"{version}":{{"defaultVersion":true,"marName":"{mar_model_name}.mar","minWorkers":1,"maxWorkers":5,"batchSize":1,"maxBatchDelay":5000,"responseTimeout":120}}}}}}}}
"""

    # write to artifact dir
    properties_path = os.path.join(cifar_config.path, 'config.properties')
    with open(properties_path, "w") as f:
      f.write(config_properties)

    torchserve_dockerfile_str = f"""FROM pytorch/torchserve:0.3.0-cpu

RUN pip install --upgrade pip
RUN pip install grpcio==1.32.0
RUN pip install pytorch-lightning

COPY config.properties /home/model-server/config.properties
COPY {mar_model_name}.mar /home/model-server/model-store/
"""
    # write to artifact dir
    dockerfile_path = os.path.join(cifar_config.path, 'Dockerfile')
    with open(dockerfile_path, "w") as f:
      f.write(torchserve_dockerfile_str)



### Define the `train` component

The train component will take as input the `Dataset` generated in the preproc component above, using that as the data source; and write its training data to the `Model` artifact's GCSFuse path.  That means that the trained model info is in GCS.  (However, it's not yet in a format that TB can handle).

This component is an adapted version of `examples/cifar10/cifar10_pytorch.py`.  The modifications include skipping the 'mar' step, which is factored out as a separate pipeline component.

In [12]:
@component(
    base_image=GPU_CONTAINER_URI,
    output_component_file="cifar_train.yaml",
)
def cifar_train(
    model_name: str,
    max_epochs: int,
    cifar_dataset: Input[Dataset],
    cifar_model: Output[Model],
):

    import pytorch_lightning as pl
    import logging
    import os
    import sys

    from pytorch_pipeline.components.trainer.component import Trainer
    from argparse import ArgumentParser
    from pytorch_lightning.loggers import TensorBoardLogger
    from pytorch_lightning.callbacks import (
        EarlyStopping,
        LearningRateMonitor,
        ModelCheckpoint,
    )

    logging.getLogger().setLevel(logging.INFO)
    logging.info('datset root path: %s', cifar_dataset.path)
    logging.info('model root path: %s', cifar_model.path)
    model_output_root = cifar_model.path


    # Argument parser for user defined paths
    parser = ArgumentParser()

    parser.add_argument(
        "--tensorboard_root",
        type=str,
        default=f"{model_output_root}/tensorboard",
        help="Tensorboard Root path (default: output/tensorboard)",
    )

    parser.add_argument(
        "--checkpoint_dir",
        type=str,
        default=f"{model_output_root}/train/models",
        help="Path to save model checkpoints ",
    )

    parser.add_argument(
        "--dataset_path",
        type=str,
        default=cifar_dataset.path,
        help="Cifar10 Dataset path (default: output/processing)",
    )

    parser.add_argument(
        "--model_name",
        type=str,
        default="resnet.pth",
        help="Name of the model to be saved as (default: resnet.pth)",
    )

    sys.argv = sys.argv[:1]

    logging.info('datset root path: %s', cifar_dataset.path)
    logging.info('model root path: %s', cifar_model.path)   

    parser = pl.Trainer.add_argparse_args(parent_parser=parser)
    args = vars(parser.parse_args())
    logging.info('args after processing: %s', args)
    print('args after processing: {}'.format(args), flush=True)
 
    # Enabling Tensorboard Logger, ModelCheckpoint, Earlystopping
    lr_logger = LearningRateMonitor()
    tboard = TensorBoardLogger(f"{model_output_root}/tensorboard")

    early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=5, verbose=True)
    checkpoint_callback = ModelCheckpoint(
        dirpath=f"{model_output_root}/train/models",
        filename="cifar10_{epoch:02d}",
        save_top_k=1,
        verbose=True,
        monitor="val_loss",
        mode="min",
    )

    # Setting the trainer specific arguments
    trainer_args = {
        "logger": tboard,
        "profiler": "pytorch",
        "checkpoint_callback": True,
        "max_epochs": max_epochs,
        "callbacks": [lr_logger, early_stopping, checkpoint_callback],
        "gpus": 1
    }

    # Setting the datamodule specific arguments
    data_module_args = {"train_glob": cifar_dataset.path}

    # Initiating the training process
    logging.info("about to call the Trainer...")

    trainer = Trainer(
        module_file="cifar10_train.py",
        data_module_file="cifar10_datamodule.py",
        module_file_args=parser,
        data_module_args=data_module_args,
        trainer_args=trainer_args,
    )



### Define the 'mar' component

This version leverages Artifacts (& GCSFuse under the hood) instead of minio.


In [13]:
@component(
    base_image=CONTAINER_URI,
    output_component_file="mar.yaml",
)
def generate_mar_file(
    model_name: str,
    mar_model_name: str,
    handler: str,
    version: str,
    config_properties: str,
    cifar_model: Input[Model],
    cifar_mar: Output[Model],
):

    import logging
    import pytorch_lightning as pl
    import os
    import subprocess

    from pathlib import Path

    # from pytorch_pipeline.components.mar.mar_generation import MarGeneration

    def _validate_mar_config(mar_config):
        mandatory_args = [
            "MODEL_NAME",
            "SERIALIZED_FILE",
            "MODEL_FILE",
            "HANDLER",
            "VERSION",
            "CONFIG_PROPERTIES",
        ]
        missing_list = []
        for key in mandatory_args:
            if key not in mar_config:
                missing_list.append(key)

        if missing_list:
            logging.warning("The following Mandatory keys are missing in the config file {} ".format(
                    missing_list))
            raise Exception(
                "Following Mandatory keys are missing in the config file {} ".format(
                    missing_list))

    logging.getLogger().setLevel(logging.INFO)

    model_output_root = cifar_model.path
    mar_output_root = cifar_mar.path
    export_path = f"{mar_output_root}/model-store"
    Path(export_path).mkdir(parents=True, exist_ok=True)

    mar_config = {
        "MODEL_NAME": mar_model_name,
        "MODEL_FILE": "pytorch_pipeline/examples/cifar10/cifar10_train.py",
        # "HANDLER": "image_classifier",
        "HANDLER": handler,
        "SERIALIZED_FILE": os.path.join(
            # args["checkpoint_dir"], args["model_name"]
            f"{model_output_root}/train/models",
            model_name,
        ),
        # "VERSION": "1",
        "VERSION": version,
        # "EXPORT_PATH": args["checkpoint_dir"],
        "EXPORT_PATH": f"{cifar_mar.path}/model-store",
        # "CONFIG_PROPERTIES": "https://kubeflow-dataset.s3.us-east-2.amazonaws.com/config.properties",
        "CONFIG_PROPERTIES": config_properties,
    }
    logging.warning("mar_config: %s", mar_config)
    print(f"mar_config: {mar_config}")
    try:
        logging.info("validating config")
        _validate_mar_config(mar_config)
    # temp testing
    except Exception as e:
      logging.warning(e)
      # return

    archiver_cmd = "torch-model-archiver --force --model-name {MODEL_NAME} --serialized-file {SERIALIZED_FILE} --model-file {MODEL_FILE} --handler {HANDLER} -v {VERSION}".format(
        MODEL_NAME=mar_config["MODEL_NAME"],
        SERIALIZED_FILE=mar_config["SERIALIZED_FILE"],
        MODEL_FILE=mar_config["MODEL_FILE"],
        HANDLER=mar_config["HANDLER"],
        VERSION=mar_config["VERSION"],
    )
    if "EXPORT_PATH" in mar_config:
        archiver_cmd += " --export-path {EXPORT_PATH}".format(
            EXPORT_PATH=mar_config["EXPORT_PATH"]
        )

    if "EXTRA_FILES" in mar_config:
        archiver_cmd += " --extra_files {EXTRA_FILES}".format(
            EXTRA_FILES=mar_config["EXTRA_FILES"]
        )

    if "REQUIREMENTS_FILE" in mar_config:
        archiver_cmd += " -r {REQUIREMENTS_FILE}".format(
            REQUIREMENTS_FILE=mar_config["REQUIREMENTS_FILE"]
        )

    print("Running Archiver cmd: ", archiver_cmd)
    logging.warning("archiver command: %s", archiver_cmd)

    try:
        return_code = subprocess.Popen(archiver_cmd, shell=True).wait()
        if return_code != 0:
            error_msg = (
                "Error running command {archiver_cmd} {return_code}".format(
                    archiver_cmd=archiver_cmd, return_code=return_code
                )
            )
            print(error_msg)
    except Exception as e:
        logging.warning(e)
        # raise Exception("Unable to create mar file: {error_msg}".format(error_msg=error_msg))


### Define the component to build a torchserve docker image

This component uses the results of the 'config' component.


In [14]:
@component(
    base_image="gcr.io/deeplearning-platform-release/tf2-gpu.2-3:latest",
    output_component_file="build_image.yaml",
)
def build_torchserve_image(
    model_name: str,
    cifar_mar: Input[Model],
    cifar_config: Input[Artifact],
    project: str,
)-> NamedTuple(
    "Outputs",
    [
        ("serving_container_uri", str), 
    ],
):

    from datetime import datetime
    import logging
    import os

    # from pathlib import Path
    # import subprocess
    import google.auth
    from google.cloud.devtools import cloudbuild_v1

    logging.getLogger().setLevel(logging.INFO)
    credentials, project_id = google.auth.default()
    client = cloudbuild_v1.services.cloud_build.CloudBuildClient()

    mar_model_name = f"{model_name}.mar"
    build_version = datetime.now().strftime("%Y%m%d%H%M%S")

    dockerfile_path = os.path.join(cifar_config.path, 'Dockerfile')
    gs_dockerfile_path = dockerfile_path.replace("/gcs/", "gs://")
    config_prop_path = os.path.join(cifar_config.path, 'config.properties')
    gs_config_prop_path = config_prop_path.replace("/gcs/", "gs://")

    export_path = f"{cifar_mar.path}/model-store"
    model_path = os.path.join(export_path, mar_model_name)
    gs_model_path = model_path.replace("/gcs/", "gs://")
    logging.warning("gs_model_path: %s", gs_model_path)
    # config_properties_path = (
    #     "/gcs/aju-pipelines/pytorch_temp/cifar_config/config.properties"
    # )
    image_uri = f"gcr.io/{project}/torchservetest:{build_version}"
    logging.info("image uri: %s", image_uri)

    build = cloudbuild_v1.Build(
        images=[image_uri]
    )
    build.steps = [
        {
            "name": "gcr.io/cloud-builders/gsutil",
            "args": [
                "cp",
                gs_config_prop_path,
                "config.properties",
            ],
        },
        {
            "name": "gcr.io/cloud-builders/gsutil",
            "args": ["cp", f"{gs_model_path}", f"{mar_model_name}"],
        },
        {
            "name": "gcr.io/cloud-builders/gsutil",
            "args": [
                "cp",
                gs_dockerfile_path,
                "Dockerfile",
            ],
        },
        {
          "name": 'gcr.io/cloud-builders/docker',
          "args": [ 'build', '-t', image_uri, '.' ]
        }
    ]
    operation = client.create_build(project_id=project, build=build)
    print("IN PROGRESS:")
    print(operation.metadata)

    result = operation.result()
    # Print the completed status
    print("RESULT:", result.status)
    return(image_uri,)



### Build Tensorboard (on CAIP Notebooks) component

(Work in progress)

In [15]:
@component(
    base_image="gcr.io/deeplearning-platform-release/tf2-gpu.2-3:latest",
    output_component_file="build_tb_image.yaml",
    packages_to_install=['google-cloud-notebooks']
)
def build_tb_image(
    project: str,
    project_num: str,
    zone: str,
    cifar_model: Input[Model],
    training_path_info: str,
    tb_config: Output[Artifact],
) -> NamedTuple("Outputs", [("tb_serving_container_uri", str),],):

    from datetime import datetime
    import logging
    import os

    from pathlib import Path
    import subprocess
    import google.auth
    from google.cloud.devtools import cloudbuild_v1
    from google.cloud import notebooks

    Path(tb_config.path).mkdir(parents=True, exist_ok=True)
    Path(training_path_info).mkdir(parents=True, exist_ok=True)

    logging.getLogger().setLevel(logging.INFO)
    credentials, project_id = google.auth.default()
    client = cloudbuild_v1.services.cloud_build.CloudBuildClient()

    # mar_model_name = f"{model_name}.mar"
    build_version = datetime.now().strftime("%Y%m%d%H%M%S")

    # model_path = os.path.join(cifar_model.path, 'tensorboard')
    model_path = cifar_model.path
    model_path = model_path.replace("/gcs/", "gs://")
    logging.info("model_path: %s", model_path)

    dockerfile_str = f"""
FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-3:latest

#RUN pip3 install torch_tb_profiler

RUN mkdir summaries
COPY summaries/* /summaries/

ENTRYPOINT ["tensorboard", "--bind_all", "--port", "8080", "--logdir", "/summaries"]
#ENTRYPOINT ["tensorboard", "--bind_all", "--port", "8080", "--logdir", "{model_path}"]

"""

    # write to artifact dir
    dockerfile_path = os.path.join(tb_config.path, "Dockerfile")
    with open(dockerfile_path, "w") as f:
        f.write(dockerfile_str)

    image_uri = f"gcr.io/{project}/tb_test:{build_version}"
    logging.info("image uri: %s", image_uri)

    build = cloudbuild_v1.Build(images=[image_uri])
    build.steps = [
        {
            "name": "ubuntu",
            "args": [
                "mkdir",
                "-p",
                model_path,
                "summaries",
            ],
        },
        {
            "name": "gcr.io/cloud-builders/gsutil",
            "args": [
                "cp",
                "-r",
                model_path,
                "summaries",
            ],
        },
        {
            "name": "gcr.io/cloud-builders/gsutil",
            "args": [
                "cp",
                dockerfile_path.replace("/gcs/", "gs://"),
                "Dockerfile",
            ],
        },
        {
            "name": "gcr.io/cloud-builders/docker",
            "args": ["build", "-t", image_uri, "."],
        },
    ]
    operation = client.create_build(project_id=project, build=build)
    print("IN PROGRESS:")
    print(operation.metadata)

    result = operation.result()
    # Print the completed status
    print("RESULT:", result.status)

    logging.warning("creating tensorboard notebook instance")
    print("creating tensorboard notebook instance")

    # create tb 'notebook'
    parent = f"projects/{project_num}/locations/{zone}"
    notebooks_client = notebooks.NotebookServiceClient()
    notebook_id = f"tb-nb{build_version}"
    container_image = notebooks.ContainerImage(
        {"repository": f"gcr.io/{project}/tb_test", "tag": f"{build_version}"}
    )
    # Define instance information
    instance = notebooks.Instance(
        {
            "name": notebook_id,
            "machine_type": "n1-standard-4",
            "container_image": container_image,
            "no_public_ip": False,
            "metadata": {"agent-health-check-interval-seconds": "0"},
            "service_account": "default",
            # "service_account": "467744782358@developer.gserviceaccount.com",
        }
    )
    # Create a new AI Platform Notebook Instance
    nb1 = notebooks_client.create_instance(
        notebooks.CreateInstanceRequest(
            {
                "parent": parent,
                "instance_id": notebook_id,
                "instance": instance,
            }
        )
    )
    # List operation status
    print(nb1.operation)

    return (image_uri,)


...

## Define and run the pipeline

Define a pipeline that uses these components. 
**Edit the GPU type** depending upon what quota you have available.

Define some constants

In [16]:
from datetime import datetime
ts = datetime.now().strftime("%Y%m%d%H%M%S")
MODEL_NAME = f'resnet{ts}'
PORT = 8080
MAR_MODEL_NAME = 'cifar10'

In [17]:
print(MODEL_NAME)

resnet20210525214733


In [18]:
PROJECT_NUMBER = '560224572293'
ZONE = 'us-central1-c'
TRAINING_PATH_INFO = os.path.join(PIPELINE_ROOT, 'training_path', ts).replace("gs://", "/gcs/")
print(TRAINING_PATH_INFO)

/gcs/cloud-ai-platform-2f444b6a-a742-444b-b91a-c7519f51bd77/pipeline_root/rthallam/training_path/20210525214733


In [19]:

@dsl.pipeline(
    name="pytorch-test-pipeline",
    pipeline_root=PIPELINE_ROOT,
)
def pytorch_cifar_pipeline(
    project: str = PROJECT_ID,
    project_num: str = PROJECT_NUMBER,
    zone: str = ZONE,
    model_name: str = "resnet.pth",
    model_display_name: str = MODEL_NAME,
    max_epochs: int = 1,
    mar_model_name: str = MAR_MODEL_NAME,
    handler: str = "image_classifier",
    version: str = "1.0",
    port: int = PORT,
    training_path_info: str = TRAINING_PATH_INFO,
    # config_properties: str = "https://kubeflow-dataset.s3.us-east-2.amazonaws.com/config.properties",
    config_properties: str = 'https://storage.googleapis.com/aju-pipelines/pytorch_temp/cifar_config/config.properties',
):
    cifar_config_task = cifar_config(mar_model_name, version, port)
    cifar_preproc_task = cifar_preproc()

    cifar_train_task = cifar_train(
        model_name=model_name,
        max_epochs=max_epochs,
        cifar_dataset=cifar_preproc_task.outputs["cifar_dataset"],
    ).set_gpu_limit(1).set_memory_limit('32G')
    cifar_train_task.add_node_selector_constraint(
        "cloud.google.com/gke-accelerator", "nvidia-tesla-v100"
        # "cloud.google.com/gke-accelerator",
        # "nvidia-tesla-k80",
    )

    cifar_mar_task = generate_mar_file(
        model_name,
        mar_model_name,
        handler,
        version,
        config_properties,
        cifar_train_task.outputs["cifar_model"],
    )

    build_image_task = build_torchserve_image(
        mar_model_name, cifar_mar_task.outputs["cifar_mar"], 
        cifar_config_task.outputs['cifar_config'],
        project
    )

    model_upload_op = gcc_aip.ModelUploadOp(
        project=project,
        display_name=model_display_name,
        # artifact_uri=WORKING_DIR,
        serving_container_image_uri=build_image_task.outputs['serving_container_uri'],
        serving_container_predict_route="/predictions/{}".format(MAR_MODEL_NAME),
        serving_container_health_route="/ping",
        serving_container_ports=[PORT]        
        # serving_container_environment_variables={"NOT_USED": "NO_VALUE"},
    )
    
    endpoint_create_op = gcc_aip.EndpointCreateOp(
        project=project,
        display_name=model_display_name,
    )

    model_deploy_op = gcc_aip.ModelDeployOp(
        project=project,
        endpoint=endpoint_create_op.outputs["endpoint"],
        model=model_upload_op.outputs["model"],
        deployed_model_display_name=model_display_name,
        machine_type="n1-standard-4",
    )

    build_tb_image_task = build_tb_image(
        project, project_num, zone,
        cifar_train_task.outputs["cifar_model"],
        training_path_info
    )


In [20]:
from kfp.v2 import compiler as v2compiler
v2compiler.Compiler().compile(pipeline_func=pytorch_cifar_pipeline,
                              package_path='pytorch_test_pipeline_spec.json')

In [21]:
from kfp.v2.google.client import AIPlatformClient  # noqa: F811

api_client = AIPlatformClient(
    project_id=PROJECT_ID, 
    region=REGION, 
    )

In [22]:
result = api_client.create_run_from_job_spec(
    job_spec_path="pytorch_test_pipeline_spec.json",
    pipeline_root=PIPELINE_ROOT,
    # enable_caching=False,
    parameter_values={
        "model_name": "resnet.pth", "max_epochs": 5,
        "project": PROJECT_ID, "model_display_name": MODEL_NAME,
    },
)

In [32]:
%%writefile instances.json
{
  "instances": [
    {
      "data": {
        "b64": "iVBORw0KGgoAAAANSUhEUgAAACAAAAAgCAIAAAD8GO2jAAAAB3RJTUUH2AoYFB0lQuPgxAAAB/ZJREFUSImVVlmPplURruWcd+ttpnt6mZ4JdsMwC0OUxAXcgka58Xf4O/wnmsi1N8QQ3EJMBA1ookNUZJh9pqe3r7/9Pe97tiovPiBR4cK6q0rqqarnpE49+OOf/FwksMHClsZaa5kIAJTIWGuZGQFSTEFFM+Q+uaRdiDknEWXIIfg+SMqQBVUg5jRzLgQfg/cp9yLG2FIyISmSISJEYkYkRCBEVFUFIGNQNQMIE+REJABQlmbzwpL33cNHJ4pGowJjBiBiYkNGETJmIcPEbJgsISEiAgACExHRwlNVAFEARRJiQCRCRMw5A8jm5vmqssxEzAAAAERIhITEzMxskBIDEBljmBmJEUAVgBAXCUxIGgu2WYQIiYmVRCTnfHJyOhgAALFhEUkxSc6AiIilIYk5oVJVs7HABlQFEYmYiEABERdDWKbVxpJ6wkwozMS8IBNVbOdUhRa9ICIiMrNlXl9tGks1g9n/0vrpYOpcIlOJKjIQMSEQLQbAqjAltdK3ZXU+MomKKBITKaOQNRxSJlJmQEOsZASt4YqgKcuMbK7vra4tmydH45AxCUkWUGUmIhBkjdlSpjSvDAoxMCcJiYQJgVlQVNUYQ4pIWSHX1rDQfDQ3wlVZz5M3qzWWl9bWVuuDo9F0FoCKJJSTEBKyBYAkXvsuBi4ay4qsZJTAEoAAKBtSAQaqyWyfr1fr8vxK9frPft+snt945gY6MAVl1bSzbrfPX37w4GTURuG673OM6lXJUFnX81HyEY0qEBVFSczeZwRRyIAIxKrAhNYYYo25G89O1y5eqBuyUzVLdanOqfaVtS9cuXg4mI3mCZar0cSNughEOefJ1K1s7gmRiIgqIhZFwaRIklJSwRhzH2Xug+Rec3fzle89/9z+vI32tDW9DylKWVYoqSnM/qX1TReHoxkKdCG2vdqlZm//SquVA4OaQHSxHIhYFLYoTIxCRBpyCJLEZK2rld2Ts64pytKw6UMKQTULlIrgqgovLNslU6+s2LYL47NxJIOIrevs2nIGAM0xZkJUQkVEQFVAJMNcWfAee0/ex3nXr2yUdVkZEMxJCHJOkJTUiK0z1WCq5vLOhpul5MO4cz4BFHMiY4mwYATN3mfDEZCQRCWFLvSTFHJjq42V5cOn0+ngxJI1IIAgAImo8H3u3PTC5jKxqOjhwcFkNN7evFg0ZZNy2/ddG9EUzWpz+fK21XT38fHMS++70cnw+PBgcHbYzmYXN7dv3rh+94P3bn/08OpL3zQiwsyICIrD0eTDf97Z3t7cvbSTxLz7h/cOjoY3rl27eGm7rJraSm2Ladu58TyuF+dWmjA8Onp8PByM3NTNu861MzedfXwy+OjWB/OzietS1zkDACklY0wWIq5SxsdPzgaDdmd3d2NjS6kaj4cpzJLIdOae2XtOtRhOxqcHT6Sb9167oKHtU4gpqo8UEgefummLfdrZ2d2/8pwJMagKEc3m/Z//covZeJ/LAre2Ni+N3Zf2rxCo72YPHj46OX7gulTZ8z6lGB0lr2gzFwm4S+CShkw+Ux91aXWdiu75F29u7W6TZFEFY8zt2w/+dusftjAh+aUVu31xbWW1RiIiOxhMh2PXLG/0XkeTcdvNfewAxRpEiSAxS4h+nrtRdkPpxqzh4rN7ezevU2lNVZUpJUScTV1RVM1SNRpPs+Sub+ft7J13/mqp8j75GPso1tqyymhy3dQ76+cub13647vvPzm8P5q1s9ksdC52wYLO/Wj/tR9u7O60zpuQOwLWLGvLSzeuP39ha+vw2B0cpV//5oOP7zxy0wTaLjVLq01p+zanfr0xIiG2848P7r9/9vbdu/fmcyeqokjcFNb64C7t7V598RpXVkQMQOn79ODO/ZmbXHvhquty26XRcNp3XUrBGj23tqKpf/T44Wg0dO2MIU5GZ851WbKqLFaaABAQJIKxVJWvfP/Vje2N+byvSjauk9Tnwdnkws65BHrv/vF4Mu19Gzo/ODsanB02je1no8OnR94nBDZMIspcVFUhEvvgJAsAEChI70J+/stf+fp3vqsFVDUxkZnOpr7rysasrC23nR48Ob59+1+D4yfT4aR1o5TblDxnZCoKu9Q0K0mi9z7nHGMWiSr6ySlGUElVU/7gR69d2N0ejWdVAYxoQpLxZOY7V1fnfvfbt3/15lvDwZFmDyLEwkZqWwBVkiEESXGW1AMqAqYcATKA4CfXG5PItevPfvvVlyPlqkDIgiLmwb1x3zoCfeuXf3rjF2+enT5c6AuwAIAgmDMoCCAoJgFBVABFRABUXfysgAgCCMa+/K1vrK8vHY9aa1Uyioh5+GgwHg5Oj4+nQzcbtQUjoaiAqgAgAgEiYFYQAEFUVAIFgEWZT9ARMaW0vr310le/lsUTRAJEECYwhydPh2fj4XDST52IEJBgVgJQBABAAFDUTx3FRegz+1TcgEi4evXq3v61KF3JJAqkwKqEhOPJJOVcVLi0YgEEAPU/UT5Dg8+PLyQLXb3yQl2ek0CYrWZCZQQ2IYSUovdhOnwqbsiYPwXSL8D6XFM09Yf3Tn/6+hsuj0kKUsqSVdT43q+vb8SYUz9uuzMQ0IXswv9udvGk/xtfECVY3rpz9Pen3uc5JVskSTHmLMYWtm07a4uyWPLcSE4A6f9sH1QUGXKIgt77vgAMSVJKKSXjWuecMyaDGoQSlAEzgH4x3Z9XQJUlQHvq53OVXmk5GSuaAeXfrDxOutzfcVMAAAAASUVORK5CYII="
      }
    }
  ]
}

Writing instances.json


In [45]:
!gcloud ai endpoints predict 3409277694477598720 --json-request=instances.json --region=us-central1

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
ERROR: (gcloud.ai.endpoints.predict) HttpError accessing <https://us-central1-prediction-aiplatform.googleapis.com/v1/projects/rthallam-demo-project/locations/us-central1/endpoints/3409277694477598720:predict?alt=json>: response: <{'vary': 'Origin, X-Origin, Referer', 'content-type': 'application/json; charset=UTF-8', 'content-encoding': 'gzip', 'date': 'Fri, 21 May 2021 16:54:07 GMT', 'server': 'scaffolding on HTTPServer2', 'cache-control': 'private', 'x-xss-protection': '0', 'x-frame-options': 'SAMEORIGIN', 'x-content-type-options': 'nosniff', 'transfer-encoding': 'chunked', 'status': 503}>, content <{
  "error": {
    "code": 503,
    "message": "Unable to process the request when processing endpoint_id: 3409277694477598720, deployed_model_id: 647867435457708032.",
    "status": "UNAVAILABLE"
  }
}
>
This may be due to network connectivity issues. Please check your network settings, and the status of the serv

In [39]:
from typing import Dict

from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instance_dict: Dict,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    # The format of each instance should conform to the deployed model's prediction input schema.
    instance = json_format.ParseDict(instance_dict, Value())
    instances = [instance]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # The predictions are a google.protobuf.Value representation of the model's predictions.
    predictions = response.predictions
    for prediction in predictions:
        print(" prediction:", dict(prediction))


# [END aiplatform_predict_custom_trained_model_sample]

In [43]:
instance = {
  "instance_key_1":
    {
        "b64": "iVBORw0KGgoAAAANSUhEUgAAACAAAAAgCAIAAAD8GO2jAAAAB3RJTUUH2AoYFB0lQuPgxAAAB/ZJREFUSImVVlmPplURruWcd+ttpnt6mZ4JdsMwC0OUxAXcgka58Xf4O/wnmsi1N8QQ3EJMBA1ookNUZJh9pqe3r7/9Pe97tiovPiBR4cK6q0rqqarnpE49+OOf/FwksMHClsZaa5kIAJTIWGuZGQFSTEFFM+Q+uaRdiDknEWXIIfg+SMqQBVUg5jRzLgQfg/cp9yLG2FIyISmSISJEYkYkRCBEVFUFIGNQNQMIE+REJABQlmbzwpL33cNHJ4pGowJjBiBiYkNGETJmIcPEbJgsISEiAgACExHRwlNVAFEARRJiQCRCRMw5A8jm5vmqssxEzAAAAERIhITEzMxskBIDEBljmBmJEUAVgBAXCUxIGgu2WYQIiYmVRCTnfHJyOhgAALFhEUkxSc6AiIilIYk5oVJVs7HABlQFEYmYiEABERdDWKbVxpJ6wkwozMS8IBNVbOdUhRa9ICIiMrNlXl9tGks1g9n/0vrpYOpcIlOJKjIQMSEQLQbAqjAltdK3ZXU+MomKKBITKaOQNRxSJlJmQEOsZASt4YqgKcuMbK7vra4tmydH45AxCUkWUGUmIhBkjdlSpjSvDAoxMCcJiYQJgVlQVNUYQ4pIWSHX1rDQfDQ3wlVZz5M3qzWWl9bWVuuDo9F0FoCKJJSTEBKyBYAkXvsuBi4ay4qsZJTAEoAAKBtSAQaqyWyfr1fr8vxK9frPft+snt945gY6MAVl1bSzbrfPX37w4GTURuG673OM6lXJUFnX81HyEY0qEBVFSczeZwRRyIAIxKrAhNYYYo25G89O1y5eqBuyUzVLdanOqfaVtS9cuXg4mI3mCZar0cSNughEOefJ1K1s7gmRiIgqIhZFwaRIklJSwRhzH2Xug+Rec3fzle89/9z+vI32tDW9DylKWVYoqSnM/qX1TReHoxkKdCG2vdqlZm//SquVA4OaQHSxHIhYFLYoTIxCRBpyCJLEZK2rld2Ts64pytKw6UMKQTULlIrgqgovLNslU6+s2LYL47NxJIOIrevs2nIGAM0xZkJUQkVEQFVAJMNcWfAee0/ex3nXr2yUdVkZEMxJCHJOkJTUiK0z1WCq5vLOhpul5MO4cz4BFHMiY4mwYATN3mfDEZCQRCWFLvSTFHJjq42V5cOn0+ngxJI1IIAgAImo8H3u3PTC5jKxqOjhwcFkNN7evFg0ZZNy2/ddG9EUzWpz+fK21XT38fHMS++70cnw+PBgcHbYzmYXN7dv3rh+94P3bn/08OpL3zQiwsyICIrD0eTDf97Z3t7cvbSTxLz7h/cOjoY3rl27eGm7rJraSm2Ladu58TyuF+dWmjA8Onp8PByM3NTNu861MzedfXwy+OjWB/OzietS1zkDACklY0wWIq5SxsdPzgaDdmd3d2NjS6kaj4cpzJLIdOae2XtOtRhOxqcHT6Sb9167oKHtU4gpqo8UEgefummLfdrZ2d2/8pwJMagKEc3m/Z//covZeJ/LAre2Ni+N3Zf2rxCo72YPHj46OX7gulTZ8z6lGB0lr2gzFwm4S+CShkw+Ux91aXWdiu75F29u7W6TZFEFY8zt2w/+dusftjAh+aUVu31xbWW1RiIiOxhMh2PXLG/0XkeTcdvNfewAxRpEiSAxS4h+nrtRdkPpxqzh4rN7ezevU2lNVZUpJUScTV1RVM1SNRpPs+Sub+ft7J13/mqp8j75GPso1tqyymhy3dQ76+cub13647vvPzm8P5q1s9ksdC52wYLO/Wj/tR9u7O60zpuQOwLWLGvLSzeuP39ha+vw2B0cpV//5oOP7zxy0wTaLjVLq01p+zanfr0xIiG2848P7r9/9vbdu/fmcyeqokjcFNb64C7t7V598RpXVkQMQOn79ODO/ZmbXHvhquty26XRcNp3XUrBGj23tqKpf/T44Wg0dO2MIU5GZ851WbKqLFaaABAQJIKxVJWvfP/Vje2N+byvSjauk9Tnwdnkws65BHrv/vF4Mu19Gzo/ODsanB02je1no8OnR94nBDZMIspcVFUhEvvgJAsAEChI70J+/stf+fp3vqsFVDUxkZnOpr7rysasrC23nR48Ob59+1+D4yfT4aR1o5TblDxnZCoKu9Q0K0mi9z7nHGMWiSr6ySlGUElVU/7gR69d2N0ejWdVAYxoQpLxZOY7V1fnfvfbt3/15lvDwZFmDyLEwkZqWwBVkiEESXGW1AMqAqYcATKA4CfXG5PItevPfvvVlyPlqkDIgiLmwb1x3zoCfeuXf3rjF2+enT5c6AuwAIAgmDMoCCAoJgFBVABFRABUXfysgAgCCMa+/K1vrK8vHY9aa1Uyioh5+GgwHg5Oj4+nQzcbtQUjoaiAqgAgAgEiYFYQAEFUVAIFgEWZT9ARMaW0vr310le/lsUTRAJEECYwhydPh2fj4XDST52IEJBgVgJQBABAAFDUTx3FRegz+1TcgEi4evXq3v61KF3JJAqkwKqEhOPJJOVcVLi0YgEEAPU/UT5Dg8+PLyQLXb3yQl2ek0CYrWZCZQQ2IYSUovdhOnwqbsiYPwXSL8D6XFM09Yf3Tn/6+hsuj0kKUsqSVdT43q+vb8SYUz9uuzMQ0IXswv9udvGk/xtfECVY3rpz9Pen3uc5JVskSTHmLMYWtm07a4uyWPLcSE4A6f9sH1QUGXKIgt77vgAMSVJKKSXjWuecMyaDGoQSlAEzgH4x3Z9XQJUlQHvq53OVXmk5GSuaAeXfrDxOutzfcVMAAAAASUVORK5CYII="
    }
}

predict_custom_trained_model_sample(
    project="560224572293",
    endpoint_id="3409277694477598720",
    location="us-central1",
    instance_dict=instance
)

ServiceUnavailable: 503 Unable to process the request when processing endpoint_id: 3409277694477598720, deployed_model_id: 6322402965944532992.